In [3]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus
import psycopg2

In [4]:
# Load environment variables from .env
load_dotenv()

True

In [5]:
# Fetch database credentials from .env
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

# Encode the password to handle special characters
password_encoded = quote_plus(DB_PASSWORD)

# Use only this connection string (DO NOT create another one)
connection_url = f'postgresql://{DB_USER}:{password_encoded}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Use this engine throughout the script
engine = create_engine(connection_url)

In [6]:
# Establish a connection using psycopg2
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

print("Connection to PostgreSQL successful!")

Connection to PostgreSQL successful!


In [7]:
# Testing the connection by fetching table names
query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
tables = pd.read_sql(query, engine)
print(tables)

    table_name
0  credit_data


In [8]:
# Ensuring all columns are correctly stored in PostgreSQL
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'credit_data';
""" 
columns_info = pd.read_sql(query, engine)
columns_info

,column_name,data_type
0,id,bigint
1,limit_bal,bigint
2,sex,bigint
3,education,bigint
4,marriage,bigint
5,age,bigint
6,pay_0,bigint
7,pay_2,bigint
8,pay_3,bigint
9,pay_4,bigint


In [9]:
# Checking all the dataset fully loaded
query = """
SELECT COUNT(*) FROM credit_data;
"""
total_records = pd.read_sql(query, engine)
total_records

,count
0,29999


In [10]:
# Checking for any missing values

# Get column names dynamically from PostgreSQL
query_columns = """
SELECT column_name 
FROM information_schema.columns 
WHERE table_name = 'credit_data';
"""
columns = pd.read_sql(query_columns, engine)['column_name'].tolist()

# Generate SQL query dynamically
query_missing = "SELECT " + ", ".join(
    [f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS {col}_missing" for col in columns]
) + " FROM credit_data;"

# Execute the query
missing_values = pd.read_sql(query_missing, engine)
missing_values

,id_missing,limit_bal_missing,sex_missing,education_missing,marriage_missing,age_missing,pay_0_missing,pay_2_missing,pay_3_missing,pay_4_missing,...,high_credit_limit_missing,credit_limit_category_missing,limit_bal_group_missing,pay_amt_group_missing,pay_amt1_group_missing,pay_amt2_group_missing,pay_amt3_group_missing,pay_amt4_group_missing,pay_amt5_group_missing,pay_amt6_group_missing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Checking for any duplicate values
query = """
SELECT COUNT(*) - COUNT(DISTINCT (id, limit_bal, sex, education, marriage, age, pay_0, pay_2, pay_3, pay_4, bill_amt1, bill_amt2, bill_amt3, bill_amt4, bill_amt5, bill_amt6, pay_amt1, pay_amt2, pay_amt3, pay_amt4, pay_amt5, pay_amt6, default_payment_next_month, high_credit_limit, credit_limit_category, limit_bal_group, pay_amt_group, pay_amt1_group, pay_amt2_group, pay_amt3_group, pay_amt4_group, pay_amt5_group, pay_amt6_group)) AS duplicate_count 
FROM credit_data;
"""
duplicates = pd.read_sql(query, engine)
duplicates

,duplicate_count
0,0


In [12]:
# Analyzing patterns and risk trends

query = """
SELECT
    MIN(limit_bal) AS min_limit_bal,
    MAX(limit_bal) AS max_limit_bal,
    AVG(limit_bal) AS avg_limit_bal,
    STDDEV(limit_bal) AS std_limit_bal,

    MIN(age) AS min_age,
    MAX(age) AS max_age,
    AVG(age) AS avg_age,
    STDDEV(age) AS std_age
FROM credit_data;
"""
overview_data = pd.read_sql(query, engine)
overview_data

,min_limit_bal,max_limit_bal,avg_limit_bal,std_limit_bal,min_age,max_age,avg_age,std_age
0,10000,1000000,167475.238508,129740.283364,21,75,35.484049,9.214633


In [13]:
# Analyzing distribution of credit limit
query = """
SELECT credit_limit_category, COUNT(*) AS count
FROM credit_data
GROUP BY credit_limit_category
ORDER BY credit_limit_category;
"""
credit_distribution = pd.read_sql(query, engine)
credit_distribution

,credit_limit_category,count
0,High Credit Limit,7643
1,Low Credit Limit,7676
2,Medium Credit Limit,7614
3,Very High Credit Limit,7066


In [14]:
# Analysing the late payments per category
query = """
SELECT
    pay_0, COUNT(*) AS count_pay_0,
    pay_2, COUNT(*) AS count_pay_2,
    pay_3, COUNT(*) AS count_pay_3,
    pay_4, COUNT(*) AS count_pay_4,
    pay_5, COUNT(*) AS count_pay_5,
    pay_6, COUNT(*) AS count_pay_6
FROM credit_data
GROUP BY pay_0, pay_2, pay_3, pay_4, pay_5, pay_6
ORDER BY pay_0 DESC, pay_2 DESC, pay_3 DESC, pay_4 DESC, pay_5 DESC, pay_6 DESC;
"""
late_payments = pd.read_sql(query, engine)
late_payments

,pay_0,count_pay_0,pay_2,count_pay_2,pay_3,count_pay_3,pay_4,count_pay_4,pay_5,count_pay_5,pay_6,count_pay_6
0,8,19,7,19,6,19,5,19,4,19,3,19
1,7,9,6,9,5,9,4,9,3,9,2,9
2,6,1,5,1,4,1,4,1,3,1,2,1
3,6,2,5,2,4,2,3,2,2,2,2,2
4,6,8,5,8,4,8,3,8,2,8,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
1101,-2,8,-2,8,-2,8,-2,8,-1,8,2,8
1102,-2,28,-2,28,-2,28,-2,28,-1,28,0,28
1103,-2,49,-2,49,-2,49,-2,49,-1,49,-1,49
1104,-2,88,-2,88,-2,88,-2,88,-2,88,-1,88


Observations:
- Negative values like -2, -1 shows the advance payment
- Higher 'pay' values indicates delay in payment.

In [15]:
# Checking spending behavior 
query = """
SELECT
    AVG(bill_amt1) AS avg_bill_1, MAX(bill_amt1) AS max_bill_1, MIN(bill_amt1) AS min_bill_1, STDDEV(bill_amt1) AS std_bill_1,
    AVG(bill_amt2) AS avg_bill_2, MAX(bill_amt2) AS max_bill_2, MIN(bill_amt2) AS min_bill_2, STDDEV(bill_amt2) AS std_bill_2,
    AVG(bill_amt3) AS avg_bill_3, MAX(bill_amt3) AS max_bill_3, MIN(bill_amt3) AS min_bill_3, STDDEV(bill_amt3) AS std_bill_3,
    AVG(bill_amt4) AS avg_bill_4, MAX(bill_amt4) AS max_bill_4, MIN(bill_amt4) AS min_bill_4, STDDEV(bill_amt4) AS std_bill_4,
    AVG(bill_amt5) AS avg_bill_5, MAX(bill_amt5) AS max_bill_5, MIN(bill_amt5) AS min_bill_5, STDDEV(bill_amt5) AS std_bill_5,
    AVG(bill_amt6) AS avg_bill_6, MAX(bill_amt6) AS max_bill_6, MIN(bill_amt6) AS min_bill_6, STDDEV(bill_amt6) AS std_bill_6
FROM credit_data;
"""
spending_behavior = pd.read_sql(query, engine)
spending_behavior

,avg_bill_1,max_bill_1,min_bill_1,std_bill_1,avg_bill_2,max_bill_2,min_bill_2,std_bill_2,avg_bill_3,max_bill_3,...,min_bill_4,std_bill_4,avg_bill_5,max_bill_5,min_bill_5,std_bill_5,avg_bill_6,max_bill_6,min_bill_6,std_bill_6
0,49653.033968,298151,0,66400.993328,47451.615821,276871,0,63307.142341,45245.855362,267366,...,0,56622.941349,38674.999933,231693,0,53175.425231,37288.167172,224031,0,51950.179641


Observations:
- avg_bill values is around 50K but the standard deviation is higher than the `avg_bill` this shows spending varies in wider range
- max_bill is ~298K indicating some high spending customers

In [16]:
# Checking the correlation between default_payment_next_month and credit limit, late payments, and spending habits are related to defaults
query = """
SELECT
    pay_0, COUNT(*) FILTER (WHERE default_payment_next_month = 1) AS defaulters_count,
    pay_2, COUNT(*) FILTER (WHERE default_payment_next_month = 1) AS defaulters_count_2,
    pay_3, COUNT(*) FILTER (WHERE default_payment_next_month = 1) AS defaulters_count_3,
    pay_4, COUNT(*) FILTER (WHERE default_payment_next_month = 1) AS defaulters_count_4,
    pay_5, COUNT(*) FILTER (WHERE default_payment_next_month = 1) AS defaulters_count_5,
    pay_6, COUNT(*) FILTER (WHERE default_payment_next_month = 1) AS defaulters_count_6
FROM credit_data
GROUP BY pay_0, pay_2, pay_3, pay_4, pay_5, pay_6
ORDER BY defaulters_count DESC;
"""
correlation_analysis = pd.read_sql(query, engine)
correlation_analysis

,pay_0,defaulters_count,pay_2,defaulters_count_2,pay_3,defaulters_count_3,pay_4,defaulters_count_4,pay_5,defaulters_count_5,pay_6,defaulters_count_6
0,0,1026,0,1026,0,1026,0,1026,0,1026,0,1026
1,2,411,2,411,2,411,2,411,2,411,2,411
2,2,339,0,339,0,339,0,339,0,339,0,339
3,-1,284,-1,284,-1,284,-1,284,-1,284,-1,284
4,-2,284,-2,284,-2,284,-2,284,-2,284,-2,284
...,...,...,...,...,...,...,...,...,...,...,...,...
1101,0,0,0,0,-1,0,0,0,-1,0,0,0
1102,2,0,2,0,3,0,3,0,2,0,-1,0
1103,-1,0,0,0,0,0,2,0,2,0,0,0
1104,1,0,2,0,2,0,2,0,3,0,4,0


Observations:
- When pay_ values increases the default rate seems high
- Many defaulters had pay_0 = 0, customerswith pay_2 = 2 show high default count

In [17]:
# Churn Analysis

# Caculating overall churn rate
query = """
SELECT
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data;
"""
overall_churn_rate = pd.read_sql(query, engine)
overall_churn_rate

,total_customers,total_defaulters,churn_rate
0,29999,6636,22.12


Observations:
- 22.12% of customers default on their payments

In [18]:
# Analysing churn rate by credit limit
query = """
SELECT
    limit_bal,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data
GROUP BY limit_bal
ORDER BY churn_rate DESC;
"""
credit_churn_rate = pd.read_sql(query, engine)
credit_churn_rate

,limit_bal,total_customers,total_defaulters,churn_rate
0,327680,1,1,100.00
1,740000,2,1,50.00
2,40000,230,92,40.00
3,10000,493,197,39.96
4,20000,1976,698,35.32
...,...,...,...,...
76,540000,6,0,0.00
77,16000,2,0,0.00
78,800000,2,0,0.00
79,570000,8,0,0.00


observations:
- customers with 40000 and 10000 limit have highest default rate of ~40%
- some high-credit-limit customers (540000+) have a 0% churn rate

In [19]:
# Analyzing churn_rate by payment delays
query = """
SELECT 
    'pay_0' AS pay_category, pay_0 AS delay_level,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data 
GROUP BY pay_0 

UNION ALL

SELECT 
    'pay_2' AS pay_category, pay_2 AS delay_level,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data 
GROUP BY pay_2 

UNION ALL

SELECT 
    'pay_3' AS pay_category, pay_3 AS delay_level,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data 
GROUP BY pay_3 

UNION ALL

SELECT 
    'pay_4' AS pay_category, pay_4 AS delay_level,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data 
GROUP BY pay_4 

UNION ALL

SELECT 
    'pay_5' AS pay_category, pay_5 AS delay_level,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data 
GROUP BY pay_5 

UNION ALL

SELECT 
    'pay_6' AS pay_category, pay_6 AS delay_level,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data 
GROUP BY pay_6 
ORDER BY pay_category, delay_level DESC;
"""
churn_payment_delays = pd.read_sql(query, engine)
churn_payment_delays

,pay_category,delay_level,total_customers,total_defaulters,churn_rate
0,pay_0,8,19,11,57.89
1,pay_0,7,9,7,77.78
2,pay_0,6,11,6,54.55
3,pay_0,5,26,13,50.00
4,pay_0,4,76,52,68.42
...,...,...,...,...,...
59,pay_6,3,184,118,64.13
60,pay_6,2,2766,1401,50.65
61,pay_6,0,16285,3069,18.85
62,pay_6,-1,5740,975,16.99


Observations:
- Customers with high payment delays like 6,7,8 have higher churn rate of 75% and above
- Customers with 1-3 months payment delay have varing churn rates
- Customers with no delays have churn rate below 20%

In [20]:
# Analyzing churn rate by spending behavior
query = """
SELECT
    CASE
        WHEN bill_amt1 + bill_amt2 + bill_amt3 + bill_amt4 + bill_amt5 + bill_amt6 < 50000 THEN 'Low Spender'
        WHEN bill_amt1 + bill_amt2 + bill_amt3 + bill_amt4 + bill_amt5 + bill_amt6 BETWEEN 50000 AND 200000 THEN 'Medium Spender'
        ELSE 'High Spender'
    END AS spending_category,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS total_defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS churn_rate
FROM credit_data
GROUP BY spending_category
ORDER BY churn_rate DESC;
"""
spending_churn_rate = pd.read_sql(query, engine)
spending_churn_rate

,spending_category,total_customers,total_defaulters,churn_rate
0,Medium Spender,8908,2053,23.05
1,Low Spender,9526,2163,22.71
2,High Spender,11565,2420,20.93


Observations:
- Medium and Low spenders have higher churn rate of 23.05% and 22.71%
- High spenders have 20.93%, may be they are more financially stable

Fraud Trend Analysis:
- Unusual repayment behavior
- Customers who pay very little despite high bills
- Credit limit overuse signals

In [25]:
# Unusual repayment behavior: Checking customers who consistently pay very little compared to their bill amount

query = """
SELECT
    id,
    bill_amt1,
    pay_amt1,
    ROUND(CAST((pay_amt1 / NULLIF(bill_amt1, 0)) * 100 AS NUMERIC), 2) AS payment_ratio,
    CASE
        WHEN (pay_amt1 / NULLIF(bill_amt1, 0)) < 0.05 THEN 'Highly Unusual Behavior'
        WHEN (pay_amt1 / NULLIF(bill_amt1, 0)) < 0.10 THEN 'Unusual Behavior'
        ELSE 'Normal Behavior'
    END AS payment_behavior
FROM credit_data
WHERE bill_amt1 > 0
ORDER BY payment_ratio ASC;
"""
unusual_payment_behavior = pd.read_sql(query, engine)

import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

print(unusual_payment_behavior)

          id  bill_amt1   pay_amt1  payment_ratio         payment_behavior
0      15543      65087      0.000           0.00  Highly Unusual Behavior
1      22813      41730      0.000           0.00  Highly Unusual Behavior
2      22872     160596      6.000           0.00  Highly Unusual Behavior
3       3790      50471      0.000           0.00  Highly Unusual Behavior
4       3791       1197      0.000           0.00  Highly Unusual Behavior
5      16347       6369      0.000           0.00  Highly Unusual Behavior
6       6568      14500      0.000           0.00  Highly Unusual Behavior
7      24035       7326      0.000           0.00  Highly Unusual Behavior
8      16342       1248      0.000           0.00  Highly Unusual Behavior
9      16340      15780      0.000           0.00  Highly Unusual Behavior
10     16338         51      0.000           0.00  Highly Unusual Behavior
11     15386        336      0.000           0.00  Highly Unusual Behavior
12      3804      21723  

In [28]:
# Grouping the payment behavior

query = """
SELECT
    payment_behavior,
    COUNT(*) AS customer_count
FROM (
    SELECT
        id,
        bill_amt1,
        pay_amt1,
        ROUND(CAST((pay_amt1 / NULLIF(bill_amt1, 0)) * 100 AS NUMERIC), 2) AS payment_ratio,
        CASE
            WHEN(pay_amt1 / NULLIF(bill_amt1, 0)) < 0.05 THEN 'Highly Unusual Behavior'
            WHEN(pay_amt1 / NULLIF(bill_amt1, 0)) < 0.10 THEN 'Unusual Behavior'
            ELSE 'Normal Behavior'
        END AS payment_behavior
    FROM credit_data
    WHERE bill_amt1 > 0
) AS subquery
GROUP BY payment_behavior
ORDER BY customer_count DESC;
"""
payment_behavior_group = pd.read_sql(query, engine)
payment_behavior_group

,payment_behavior,customer_count
0,Highly Unusual Behavior,11555
1,Normal Behavior,10381
2,Unusual Behavior,5465


In [31]:
# Checking for the default rates for each payment behavior: by counting the number of customers in each group, counting the number if defaulters, calculating the default rate

query = """
SELECT
    payment_behavior,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS default_rate
FROM (
    SELECT
        id,
        bill_amt1,
        pay_amt1,
        default_payment_next_month,
        ROUND(CAST((pay_amt1 / NULLIF(bill_amt1, 0)) * 100 AS NUMERIC), 2) AS payment_ratio,
        CASE
            WHEN (pay_amt1 / NULLIF(bill_amt1, 0)) < 0.05 THEN 'Highly Unusual Behavior'
            WHEN (pay_amt1 / NULLIF(bill_amt1, 0)) < 0.10 THEN 'Unusual Behavior'
            ELSE 'Normal Behavior'
        END AS payment_behavior
    FROM credit_data
    WHERE bill_amt1 > 0
) AS subquery
GROUP BY payment_behavior
ORDER BY default_rate DESC;
"""
payment_default_rates = pd.read_sql(query, engine)
payment_default_rates

,payment_behavior,total_customers,defaulters,default_rate
0,Highly Unusual Behavior,11555,2954,25.56
1,Unusual Behavior,5465,1188,21.74
2,Normal Behavior,10381,1851,17.83


Observations:
- Some customers with 'Highly Unusual Behavior' might be paying way more than their actual bill
- Some customers with 'Normal Behavior' might have small bills but large payment

Action:
- Combining Unusual Repayment Behavior with Credit Limit Categories
- By grouping both repayment behavior & credit limit, to see if default trends differ across credit levels 

In [32]:
# Classifying repayment behavior within each credit limit group

query = """
SELECT
    credit_limit_category,
    payment_behavior,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) AS defaulters,
    ROUND(100.0 * SUM(CASE WHEN default_payment_next_month = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS default_rate
FROM(
    SELECT
        id,
        bill_amt1,
        pay_amt1,
        default_payment_next_month,

        CASE
            WHEN limit_bal BETWEEN 0 AND 50000 THEN 'Low Credit Limit'
            WHEN limit_bal BETWEEN 50001 AND 140000 THEN 'Medium Credit Limit'
            WHEN limit_bal BETWEEN 140001 AND 240000 THEN 'High Credit Limit'
            ELSE 'Very High Credit Limit'
        END AS credit_limit_category,

        CASE
            WHEN pay_amt1 < 100 OR (pay_amt1 / NULLIF(bill_amt1, 0)) < 0.50 THEN 'Highly Unusual Behavior'
            WHEN pay_amt1 BETWEEN 100 AND 500 OR (pay_amt1 / NULLIF(bill_amt1, 0)) < 0.10 THEN 'Unusual Behavior'
            ELSE 'Normal Behavior'
        END AS payment_behavior
    FROM credit_data
    WHERE bill_amt1 > 0
) AS subquery
GROUP BY credit_limit_category, payment_behavior
ORDER BY credit_limit_category, default_rate DESC;
"""
unusual_repayment_credit = pd.read_sql(query, engine)
unusual_repayment_credit

,credit_limit_category,payment_behavior,total_customers,defaulters,default_rate
0,High Credit Limit,Unusual Behavior,210,44,20.95
1,High Credit Limit,Highly Unusual Behavior,4934,847,17.17
2,High Credit Limit,Normal Behavior,1549,218,14.07
3,Low Credit Limit,Highly Unusual Behavior,6776,2209,32.60
4,Low Credit Limit,Unusual Behavior,85,24,28.24
5,Low Credit Limit,Normal Behavior,497,131,26.36
6,Medium Credit Limit,Unusual Behavior,144,47,32.64
7,Medium Credit Limit,Highly Unusual Behavior,6270,1589,25.34
8,Medium Credit Limit,Normal Behavior,732,131,17.90
9,Very High Credit Limit,Unusual Behavior,139,24,17.27


Observations:
- Medium Credit Limit + Unusual Behavior = 32.64
- Low Credit Limit + Highly Unusual Behavior = 32.60
- Low Credit Limit + Unusual Behavior = 28.24
- Low Credit Limit + Normal Behavior = 26.36

- Customers with Medium + Low credit limits with Unusual + Highly Unusual Repayment Behavior have the highest chance of defaulting
- Customers with Low credit limits might be struggling with replayments or engaging in risky behavior


In [34]:
unusual_repayment_credit.to_csv("../data/unusual_repayment_behavior.csv", index=False)

In [37]:
query = """
SELECT 
    id,
    ROUND(CAST(AVG(NULLIF(pay_amt1, 0) / NULLIF(bill_amt1, 0)) AS NUMERIC), 2) AS avg_payment_ratio,
    SUM(bill_amt1 + bill_amt2 + bill_amt3 + bill_amt4 + bill_amt5 + bill_amt6) AS total_bill_amt,
    SUM(pay_amt1 + pay_amt2 + pay_amt3 + pay_amt4 + pay_amt5 + pay_amt6) AS total_pay_amt,
    SUM(default_payment_next_month) AS default_history,
    ROUND(CAST(SUM(bill_amt1 + bill_amt2 + bill_amt3 + bill_amt4 + bill_amt5 + bill_amt6) / NULLIF(AVG(limit_bal), 0) AS NUMERIC), 2) AS credit_utilization
FROM credit_data
GROUP BY id;
"""
customer_payment_stats = pd.read_sql(query, engine)

In [38]:
customer_payment_stats.to_csv("../data/customer_payment_stats.csv", index=False)